In [1]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sci
import glob
import networkx as nx
import Bio.KEGG.KGML.KGML_parser as keg

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import cmapPy.pandasGEXpress.parse as cp
import cmapPy.pandasGEXpress.write_gctx as cw

python 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:27:36) [GCC 11.2.0]
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.0


# entrez to sym gene mapping

In [2]:
# mapping human gene symbol to entrez
dat=pd.read_table('data/Sym2Entrez.txt',sep='\t')
dat.columns=['sym','entrez']
dat=dat.loc[dat['entrez'].notnull()]
dat=dat.loc[dat['sym'].notnull()]
dat['entrez']=dat['entrez'].astype(int).astype(str)
s2e_dic=dat.set_index('sym')['entrez']
e2s_dic=dat.set_index('entrez')['sym']

# available cells

In [3]:
available_cells= ['A549','A375','BICR6','HT29','PC3','U251MG','MCF7']

# DepMap (cell growth) proc

In [4]:
## cell line info
cell_info=pd.read_table('data/DepMap_cell_metainfo(Model).csv',sep=',', index_col=0)
cell_info=cell_info.loc[cell_info['StrippedCellLineName'].isin(available_cells),['StrippedCellLineName','DepmapModelType','OncotreeLineage',
                                                                                 'OncotreePrimaryDisease','OncotreeSubtype']]
########################### Depmap CRISPR knockout
depmap_crispr_knockout_ori=pd.read_table('data/DepMap_CRISPR_knockout(CRISPRGeneEffect).csv',sep=',', index_col=0)
depmap_crispr_knockout=depmap_crispr_knockout_ori.loc[cell_info.index]
depmap_crispr_knockout=depmap_crispr_knockout.rename(index=cell_info['StrippedCellLineName'])
new_cols=[col.split(' (')[1].replace(')','') for col in depmap_crispr_knockout.columns]
depmap_crispr_knockout.columns=new_cols
depmap_crispr_knockout.head(2)
with open("./data_preproc/depmap_crispr_cellGrowth.pickle", 'wb') as file:
    pickle.dump(depmap_crispr_knockout, file)
    ## index: cell line
    ## columns: gene (knocked-out)
    ## values: cell viability

,1,29974,2,144568,127550,53947,51146,8086,65985,13,...,55055,11130,7789,158586,79364,440590,79699,7791,23140,26009
ModelID,,,,,,,,,,,,,,,,,,,,,
MCF7,-0.000655,0.010486,0.100446,0.044937,-0.073545,0.119995,0.083456,-0.247239,-0.063622,-0.029043,...,-0.111841,-0.788102,0.074418,0.039013,0.05574,-0.047248,-0.164099,0.168613,-0.093001,-0.413203
PC3,0.046644,-0.245051,-0.074162,-0.152578,-0.122190,0.098818,-0.018417,0.522229,0.196712,0.164371,...,-0.260278,-0.343973,-0.240239,-0.351804,-0.07842,0.168459,-0.196916,-0.309052,-0.135609,-0.507122


# LINCS (expression) proc for CRISPR KNOCK-OUT

In [5]:
sig_info_global=pd.read_table('/data/LDS-1613/Data/siginfo.txt',sep='\t')

## obtaining CG expression by knocking down/out all target genes
sig_info_save=sig_info_global.loc[(sig_info_global['pert_type'].isin(['trt_xpr']))&
                                (sig_info_global['pert_itime'].isin(['96 h']))][['sig_id','cmap_name','cell_iname']]

sig_info_save=sig_info_save.loc[sig_info_save['cell_iname'].isin(available_cells)]
sig_info_save['cmap_name']=sig_info_save['cmap_name'].map(s2e_dic)
sig_info_save=sig_info_save.loc[sig_info_save['cmap_name'].notnull()]
sig_info_save.head(2)
with open("./data_preproc/sig_info_crispr_selected_cells.pickle", 'wb') as file:
    pickle.dump(sig_info_save, file)

/tmp/ipykernel_269718/2997280273.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  sig_info_global=pd.read_table('/data/LDS-1613/Data/siginfo.txt',sep='\t')


,sig_id,cmap_name,cell_iname
541,XPRJJ001_A375_96H:BRDN0001056750:-666,23030,A375
543,XPRJJ001_A375_96H:BRDN0001053773:-666,8841,A375


In [6]:
for CELL in available_cells:
    ## expression data
    with open("./data_preproc/sig_info_crispr_selected_cells.pickle", 'rb') as file:
        sig_info_saved=pickle.load(file)
        
    ## cp.parse function: cid parameter
    sig_info_d1=sig_info_saved.loc[sig_info_saved['cell_iname']==CELL]
    sigid_to_gene=sig_info_d1.set_index('sig_id')['cmap_name']
    gene_exp=cp.parse('/data/LDS-1613/Data/level5_beta_all_n1102210x12328.gctx',
                      cid=sigid_to_gene.index).data_df.T.rename(index=sigid_to_gene)
    gene_exp_ave=gene_exp.groupby(gene_exp.index).mean()
    f'###################### {CELL} ###########################'
    #gene_exp_ave.head(3)
    gene_exp_ave.shape

    with open(f"./data_preproc/{CELL}_crispr_expression_data.pickle", 'wb') as file:
        pickle.dump(gene_exp_ave, file)
        
    ## index: gene (knocked-out)
    ## columns: gene (expression)
    ## values: gene(index)를 knock-out 시 gene(column)의 expreesion

/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


'###################### A549 ###########################'

(5020, 12328)

/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


'###################### A375 ###########################'

...

'###################### U251MG ###########################'

(5009, 12328)

/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


'###################### MCF7 ###########################'

(4183, 12328)

# LINCS (expression) proc for chemical

In [7]:
sig_info_global=pd.read_table('/data/LDS-1613/Data/siginfo.txt',sep='\t')

sig_info_save=sig_info_global.loc[sig_info_global['pert_type']=='trt_cp']
sig_info_save=sig_info_save.loc[sig_info_save['pert_itime']=='24 h']
sig_info_save=sig_info_save.loc[sig_info_save['pert_idose']=='10 uM']
sig_info_save=sig_info_save.loc[sig_info_save['cell_iname'].isin(available_cells)]
sig_info_save=sig_info_save[['sig_id','pert_id','cmap_name','cell_iname']]
sig_info_save=sig_info_save.drop_duplicates(subset=['cmap_name','cell_iname'])

sig_info_save.head()
sig_info_save['cell_iname'].value_counts()

with open("./data_preproc/sig_info_chemical_selected_cells.pickle", 'wb') as file:
    pickle.dump(sig_info_save, file)
    

/tmp/ipykernel_269718/2071785479.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  sig_info_global=pd.read_table('/data/LDS-1613/Data/siginfo.txt',sep='\t')


,sig_id,pert_id,cmap_name,cell_iname
13,LUNG001_A549_24H:BRD-K88551539:10,BRD-K88551539,CAY-10585,A549
15,POL001_PC3_24H:G19,BRD-K76674262,BRD-K76674262,PC3
16,DPK.CP001_A549_24H:BRD-K43880410:10,BRD-K43880410,pregnenolone,A549
17,DPK.CP002_MCF7_24H:BRD-A97437073:10,BRD-A97437073,rosiglitazone,MCF7
45,LUNG001_A549_24H:BRD-K75295174:10,BRD-K75295174,alisertib,A549


cell_iname
PC3       8703
MCF7      7975
A549      6742
A375      4740
HT29      4109
U251MG     112
BICR6      108
Name: count, dtype: int64

In [8]:
for CELL in available_cells:
    ## expression data
    with open("./data_preproc/sig_info_chemical_selected_cells.pickle", 'rb') as file:
        sig_info_saved=pickle.load(file)
        
    ## cp.parse function: cid parameter
    sig_info_d1=sig_info_saved.loc[sig_info_saved['cell_iname']==CELL]
    sigid_to_chem=sig_info_d1.set_index('sig_id')['cmap_name']
    gene_exp=cp.parse('/data/LDS-1613/Data/level5_beta_all_n1102210x12328.gctx',
                      cid=sigid_to_chem.index).data_df.T.rename(index=sigid_to_chem)
    f'###################### {CELL} ###########################'
    gene_exp=gene_exp.T

    with open(f"./data_preproc/{CELL}_chemical_expression_data.pickle", 'wb') as file:
        pickle.dump(gene_exp, file)

    gene_exp.head(3)
    gene_exp.shape
    ## index: gene (knocked-out)
    ## columns: gene (expression)
    ## values:  gene(column) expreesion when gene(index) knocked-out 


/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


'###################### A549 ###########################'

cid,CAY-10585,pregnenolone,alisertib,ZSTK-474,ENMD-2076,BRD-K93023739,CGP-53353,epothilone,AZD-6482,ketoconazole,...,SA-1463787,SA-1459454,BRD-K76438898,BRD-K00328666,BRD-K07008972,BRD-K66497480,BRD-K52369815,BRD-K79330383,BRD-K26453528,BRD-K20500634
rid,,,,,,,,,,,,,,,,,,,,,
10,-0.5012,-0.248903,-0.7386,0.5268,-0.9600,2.0459,0.2902,-0.16440,-0.14105,-1.43125,...,1.338846,-0.651769,0.099800,-0.067854,1.702898,-0.686809,1.056215,0.117462,-0.883265,-0.573049
100,3.1961,-0.398928,-0.4418,0.9355,0.5866,0.0425,0.0972,-0.24035,0.36235,-0.71590,...,-0.693796,0.434319,-0.232487,0.868822,0.736480,0.360794,-0.109926,0.403587,1.185163,-0.469960
1000,0.9071,-0.035685,-0.7679,-1.4658,-0.1116,-1.5104,-0.8743,-0.53575,0.49960,-0.62595,...,-1.741092,-0.371037,0.322125,-0.195443,-0.219794,-0.047082,-1.118701,-0.877124,0.413768,0.050085


(12328, 6742)

/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


...

(12328, 112)

/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
/home/jmjung/anaconda3/lib/python3.12/site-packages/cmapPy/pandasGEXpress/parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


'###################### MCF7 ###########################'

cid,rosiglitazone,vorinostat,sirolimus,BRD-K08317416,BRD-K77888550,BRD-K70200040,BRD-K55591206,chlormezanone,pyrimethamine,acetazolamide,...,BRD-K83790954,BRD-K43570134,BRD-K99913693,BRD-K72375447,BRD-K83216232,BRD-K68664718,BRD-K29799191,BRD-K67269534,BRD-K25642189,BRD-K70115829
rid,,,,,,,,,,,,,,,,,,,,,
10,0.557398,0.7605,0.5063,-0.289371,0.863679,-0.519472,0.429333,-0.192812,0.700926,0.707276,...,0.316940,0.471465,-0.119733,-1.033713,0.347456,0.361144,0.271031,-0.205138,0.590200,-0.468701
100,0.008377,1.0960,2.8979,0.247159,0.748929,-0.490710,-0.043234,-0.300173,0.234128,0.309988,...,0.503772,0.997556,0.104900,-0.354690,0.020789,0.128570,-0.243506,1.050315,-0.279267,-0.055902
1000,-0.205316,-2.2033,0.4305,0.477365,-0.265795,0.077660,0.855936,0.380761,-0.619850,0.346947,...,0.182248,-0.640578,-0.788367,-0.020036,0.100710,-0.826134,-0.087685,-0.017030,0.445100,-0.682508


(12328, 7975)